In [15]:
# check data against data model.

import pandas
import pathlib

def validate_properties(property_list):
    model = pandas.read_json(pathlib.Path.cwd() / 'data-model.json')
    for x in property_list:
        if x not in model.name.unique():
            raise Exception(f'"{x}" does not exist in data model.')

data = pandas.read_json(pathlib.Path.cwd() / 'dataset.json')

validate_properties(list(data.P.unique()))

print('all done.')

all done.
